# 🎶 Objectif 2 : Analyse des Flux Musicaux

#### Rappel

Analyse des aires de flux pour quatre caractéristiques musicales clé. L'objectif est d'identifier les périodes où il serait stratégique de sortir des morceaux qui se démarquent par leur niveau de dansabilité, d'énergie, d'acoustique, ou d'équilibre.

In [ ]:
# Import
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Chargement
df = pd.read_csv("./dataset/dataset_filtered.csv")
print(df.head())

# Vérification des valeurs manquantes
missing_values = df[['danceability', 'energy', 'acousticness', 'valence', 'released_year']].isnull().sum()
print("Valeurs manquantes par colonne :\n", missing_values[missing_values > 0])

In [ ]:
# On calcule la moyenne des colonnes pertinentes par année pour suivre les tendances annuelles
df_agg = df.groupby('released_year')[['danceability', 'energy', 'acousticness', 'valence']].mean().reset_index()
print("Données agrégées par année :")
print(df_agg.head())

In [ ]:
# Graphique en aire de flux
plt.figure(figsize=(12, 6))

plt.stackplot(df_agg['released_year'],
              df_agg['danceability'], df_agg['energy'], df_agg['acousticness'], df_agg['valence'],
              labels=['Dansabilité', 'Énergie', 'Acoustique', 'Valence'],
              alpha=0.8)
plt.legend(loc='upper left')
plt.xlabel("Année")
plt.ylabel("Moyenne des attributs")
plt.title("Évolution de la Dansabilité, Énergie, Acoustique et Valence des Musiques au Fil des Ans")
plt.show()